### Analyzing Doccuments Produced By Main Output

#### Goals

- Understand Data

- Fine Tune Any Issues with Output

- Find good ways to visualize Data
<br>
#### Data

- Data is derived from Topic Modeling and Labeling in addition to Sentiment Analysis on those topics.
<br><br>

#### Let's Start with Importing our Data

In [2]:
import pandas as pd

doc_df = pd.read_csv('./Data/output_topics.csv')

In [11]:
doc_df.head

<bound method NDFrame.head of    Topic  Count                         Name  \
0     -1     57       -1_br_34_battery_power   
1      0     81     0_br_speaker_sound_great   
2      1     75   1_great_product_works_good   
3      2     37  2_br_headphones_sound_br br   

                                         CustomName  \
0                  sturdy | quality price | quality   
1                   speaker | sound quality | audio   
2        great product | great quality | work great   
3  headphones | speakers headphones | sound quality   

                                      Representation  \
0  ['br', '34', 'battery', 'power', 'price', 'gre...   
1  ['br', 'speaker', 'sound', 'great', 'br br', '...   
2  ['great', 'product', 'works', 'good', 'great p...   
3  ['br', 'headphones', 'sound', 'br br', 'like',...   

                                             KeyBERT  \
0  ['sturdy', 'quality price', 'quality', 'recomm...   
1  ['speaker', 'sound quality', 'audio', 'bluetoo...   
2  [

In [12]:
doc_df.CustomName[0]

'sturdy | quality price | quality'

In [5]:
import re

def stripCustomName(name):
    return re.sub(r'\|', ' ', name).split()

doc_df['Name'] = doc_df['CustomName'].apply(stripCustomName)
#doc_df.drop('CustomName', axis=1, inplace=True)

print(doc_df.columns)


Index(['Topic', 'Count', 'Name', 'Representation', 'KeyBERT', 'POS', 'MMR',
       'Representative_Docs', 'doc_sentiment', 'Rep'],
      dtype='object')


In [8]:
print(doc_df.Representation[0])

doc_df.Representation[0][1]

['br', '34', 'battery', 'power', 'price', 'great', 'charge', 'good', 'hold', 'quality']


"'"

In [13]:
doc_df.Representative_Docs[0]

'["Fairly sturdy, all metal construction, it did come with wall anchors, screws & straps to secure it upright if you need to do so. No missing parts, and the fit & finish are very good. Instructions are very basic, but it\'s easy tool free assembly means a 1 sheet instruction page is fine.<br /><br />As the shelves are adjustable to wherever you want them to go, you may have extra shelves leftover depending on how and where you place them on the poles.  As I migrate my DVD collection to Blu-Ray, I\'ll shrink the space between shelves and add the extras later.<br /><br />Overall, very pleased with this product at this price point.", "I have ordered 3 packages of these over the past few weeks and all of them have been different sizes. The first 100 fit perfectly with about 1/4&#34; overhang at the top, I like that compared to the other sleeves that usually hang 1/2&#34;-3/4&#34; over the top of the record. The second 100 I received were cut incorrectly and I couldn\'t even get the record

In [14]:
doc_df.POS[0]

"['br', 'battery', 'power', 'price', 'great', 'charge', 'good', 'quality', 'place', 'charger']"

In [6]:
doc_df.KeyBERT[0]

"['sturdy', 'quality price', 'quality', 'recommend product', 'devices', 'charger', 'phones', 'usb', 'usb ports', 'product']"

In [9]:
doc_df.MMR[0]

"['br', '34', 'battery', 'power', 'price', 'great', 'charge', 'good', 'hold', 'quality']"

In [21]:
import ast
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

def is_valid_token(token):
    # Check if the token is not a two-letter word
    return len(token.text) > 2 or token.is_digit

def custom_tokenizer(string_list):
    
    word_list = ast.literal_eval(string_list)

    text = ' '.join(word_list)
    
    doc = nlp(text)
    
    return [token.text for token in doc if is_valid_token(token)]

34


In [41]:
topic_df = pd.DataFrame()

topic_df['label1'] = doc_df['KeyBERT'].apply(custom_tokenizer)
topic_df['label2'] = doc_df['MMR'].apply(custom_tokenizer)
topic_df['label3'] = doc_df['POS'].apply(custom_tokenizer)
topic_df['label4'] = doc_df['MMR'].apply(custom_tokenizer)
topic_df['sentiment'] = doc_df['doc_sentiment']
topic_df['count'] = doc_df['Count']


In [42]:
def labelUnion(label1,label2,label3,label4):
    listOfTopics = []
    label = label1 + label2 + label3 + label4
    for entries in (label):
        listOfTopics.append(set(entries))
    return listOfTopics

topic_df['label']=labelUnion(topic_df.label1,topic_df.label2,topic_df.label3,topic_df.label4)

In [43]:
topic_df.label

0    {quality, recommend, ports, power, 34, place, ...
1    {quality, better, music, speaker, sound, great...
2    {quality, super, week, fine, work, described, ...
3    {quality, headphones, listening, don, work, mu...
Name: label, dtype: object

In [44]:
topic_df.drop(['label1', 'label2', 'label3', 'label4'], axis=1, inplace=True)

In [45]:
topic_df.columns

Index(['sentiment', 'count', 'label'], dtype='object')

In [46]:
topic_df.to_csv('topics.csv')